# Linear Regression

In [2]:
# Imports
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lrex').getOrCreate()
from pyspark.ml.regression import LinearRegression

In [3]:
# Bring in training data. Notice there is a single features column thats a vector and have to transform the data.
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [5]:
# Build model
lr = LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction')

In [6]:
lrModel = lr.fit(training)

In [7]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [8]:
# Look at the training summary
training_summary = lrModel.summary

In [10]:
# Can look at r2 or rootmeansqared or most things off of this.
training_summary.r2

0.027839179518600154

In [11]:
# Should split the data into test train split. Or can do k fold
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

# Split the data
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [12]:
# Create new model off of training data
correct_moel = lr.fit(train_data)

In [13]:
test_results = correct_moel.evaluate(test_data)

In [14]:
test_results.rootMeanSquaredError

10.540321503037012

In [15]:
# Create unlabled dataset
unlabeled_data = test_data.select('features')

In [17]:
# Do the predictions by calling the transform
predictions = correct_moel.transform(unlabeled_data)
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| 1.7453403705148338|
|(10,[0,1,2,3,4,5,...|-0.8560096539539437|
|(10,[0,1,2,3,4,5,...| 3.4227229801686865|
|(10,[0,1,2,3,4,5,...| -2.740473093972807|
|(10,[0,1,2,3,4,5,...| 0.6817412011388044|
|(10,[0,1,2,3,4,5,...| 0.7781077626085546|
|(10,[0,1,2,3,4,5,...| 1.9172997025445278|
|(10,[0,1,2,3,4,5,...|  2.236918868494073|
|(10,[0,1,2,3,4,5,...|-2.6813211283771112|
|(10,[0,1,2,3,4,5,...|  2.200067856312657|
|(10,[0,1,2,3,4,5,...| 0.2867728607591444|
|(10,[0,1,2,3,4,5,...| 0.8978991128456428|
|(10,[0,1,2,3,4,5,...|  1.161588947117569|
|(10,[0,1,2,3,4,5,...| 0.6816111340708761|
|(10,[0,1,2,3,4,5,...| -1.539230812445407|
|(10,[0,1,2,3,4,5,...|  1.280003408248381|
|(10,[0,1,2,3,4,5,...|-0.7005282978513427|
|(10,[0,1,2,3,4,5,...| -0.721002972935624|
|(10,[0,1,2,3,4,5,...|-1.2946876967803405|
|(10,[0,1,2,3,4,5,...| -4.965126275108953|
+----------

## E-Commerce Code Along

In [19]:
############### Initialize #####################

# Import Spark and Create Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

# Import Model
from pyspark.ml.regression import LinearRegression

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [20]:
################ Data ############################
# Read CSV
data = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,header=True)

data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [21]:
# Build the assembler
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [22]:
# Need to transform that data and add new column called feautres
output = assembler.transform(data)

# Select the features and the column going to predict
final_data = output.select("features",'Yearly Amount Spent')

# Split the data
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [24]:
################### Model ##########################
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='Yearly Amount Spent')

# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data,)

# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.4870332126,38.919631539,0.300544880775,61.7069946791] Intercept: -1041.2923689106005


In [25]:
test_results = lrModel.evaluate(test_data)
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 9.467089403089085
MSE: 89.62578176608164


In [26]:
test_results.r2

0.984497274717527

In [ ]:
# To perform predictions

# Create unlabeled data or get unlabeled data
unlabeled_data = test_data.select('features')

# Do predictions
predictions = lrModel.transform(unlabeled_data)


